Here we wish to solve a 1d Burger's equations with Dirchlet boundary conditions. To do this, we begin by importing the useful packages.

In [1]:
using Random
using Statistics
using UniqueKronecker
using LinearAlgebra
using RLinearAlgebra
using PolynomialModelReductionDataset
using LiftAndLearn
using JLD2
const poly_dat = PolynomialModelReductionDataset
const LnL = LiftAndLearn

LiftAndLearn

Now that we have generated our testing data, lets look at the scale of the dataset. An consider the runtime of operator inference when we use the truncated SVD.

In [4]:
r = 8
(burger, options, μ, A, F, B, C, Utest, Xtest, Ytest, op_burger, Urand, Xall, Xdotall, X, R, U, Y) = load_object("test_trajectories.jld2")
size(X)

(512, 40000)

In [6]:
svd_time = @elapsed tmp = svd(X);
svd_time += @elapsed SVD_Vr =  tmp.U[:, 1:r];
soi_time = @elapsed SVD_op_inf = LnL.opinf(X, SVD_Vr, op_burger, options; U=U, Y=Y);
soi_time += @elapsed Finf_extract = UniqueKronecker.extractF(SVD_op_inf.A2u, r);
soi_time += @elapsed sXinf = burger.integrate_model(burger.tspan, SVD_Vr' * burger.IC, Utest; linear_matrix=SVD_op_inf.A[1:r, 1:r],
    control_matrix=SVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
soi_time += @elapsed sYinf = SVD_op_inf.C[1:1, 1:r] * sXinf;

# Compute errors
SVD_PE, SVD_ISE, SVD_IOE, SVD_OSE, SVD_OOE = LnL.compute_all_errors(Xtest, Ytest, sXinf, sYinf, sXinf, sYinf, SVD_Vr);
printstyled("The svd time is: ", svd_time, "\n")
printstyled("The operator inference time is: ", soi_time, "\n")
printstyled("The svd data error is: ", norm(X - SVD_Vr * SVD_Vr'* X),"\n")
printstyled("The svd state error is: ", SVD_OSE, "\n")

The svd time is: 2.9118834170000003
The operator inference time is: 8.968731084
The svd data error is: 0.606032868138546
The svd state error is: 0.005491591431340691


Run the same test with the randomizedSVD

With `Gaussian`

In [10]:
rsvd_time = @elapsed rtmp = rapproximate(RandSVD(compressor = Gaussian(compression_dim = r + 2, cardinality = Right())), X);
rsvd_time += @elapsed rSVD_Vr =  rtmp.U[:, 1:r];
roi_time = @elapsed rSVD_op_inf = LnL.opinf(X, rSVD_Vr, op_burger, options; U=U, Y=Y);
roi_time += @elapsed Finf_extract = UniqueKronecker.extractF(rSVD_op_inf.A2u, r);
roi_time += @elapsed rXinf = burger.integrate_model(burger.tspan, rSVD_Vr' * burger.IC, Utest; linear_matrix=rSVD_op_inf.A[1:r, 1:r],
    control_matrix=rSVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
roi_time += @elapsed rYinf = rSVD_op_inf.C[1:1, 1:r] * rXinf;

# Compute errors
rSVD_PE, rSVD_ISE, rSVD_IOE, rSVD_OSE, rSVD_OOE = LnL.compute_all_errors(Xtest, Ytest, rXinf, rYinf, rXinf, rYinf, rSVD_Vr);
printstyled("The total randomized svd time is: ", rsvd_time, "\n")
printstyled("The operator inference time is: ", roi_time, "\n")
printstyled("The randomized svd data error is: ", norm(X - rSVD_Vr * rSVD_Vr'* X),"\n")
printstyled("The randomized svd state error is: ", rSVD_OSE, "\n")

The total randomized svd time is: 0.04070975
The operator inference time is: 8.120055333
The randomized svd data error is: 0.8353486547390027
The randomized svd state error is: 0.006545813909844364


With `SparseSign`

In [11]:
rsvd_time = @elapsed rtmp = rapproximate(RandSVD(compressor = SparseSign(compression_dim = r + 2, cardinality = Right(), nnz = 4)), X);
rsvd_time += @elapsed rSVD_Vr =  rtmp.U[:, 1:r];
roi_time = @elapsed rSVD_op_inf = LnL.opinf(X, rSVD_Vr, op_burger, options; U=U, Y=Y);
roi_time += @elapsed Finf_extract = UniqueKronecker.extractF(rSVD_op_inf.A2u, r);
roi_time += @elapsed rXinf = burger.integrate_model(burger.tspan, rSVD_Vr' * burger.IC, Utest; linear_matrix=rSVD_op_inf.A[1:r, 1:r],
    control_matrix=rSVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
roi_time += @elapsed rYinf = rSVD_op_inf.C[1:1, 1:r] * rXinf;

# Compute errors
rSVD_PE, rSVD_ISE, rSVD_IOE, rSVD_OSE, rSVD_OOE = LnL.compute_all_errors(Xtest, Ytest, rXinf, rYinf, rXinf, rYinf, rSVD_Vr);
printstyled("The total randomized svd time is: ", rsvd_time, "\n")
printstyled("The operator inference time is: ", roi_time, "\n")
printstyled("The randomized svd data error is: ", norm(X - rSVD_Vr * rSVD_Vr'* X),"\n")
printstyled("The randomized svd state error is: ", rSVD_OSE, "\n")

The total randomized svd time is: 0.037474208
The operator inference time is: 8.078093418
The randomized svd data error is: 0.6366581866004271
The randomized svd state error is: 0.005541273518778629


with `CountSketch`

In [12]:
rsvd_time = @elapsed rtmp = rapproximate(RandSVD(compressor = CountSketch(compression_dim = r + 5,  cardinality = Right())), X);
rsvd_time += @elapsed rSVD_Vr =  rtmp.U[:, 1:r];
roi_time = @elapsed rSVD_op_inf = LnL.opinf(X, rSVD_Vr, op_burger, options; U=U, Y=Y);
roi_time += @elapsed Finf_extract = UniqueKronecker.extractF(rSVD_op_inf.A2u, r);
roi_time += @elapsed rXinf = burger.integrate_model(burger.tspan, rSVD_Vr' * burger.IC, Utest; linear_matrix=rSVD_op_inf.A[1:r, 1:r],
    control_matrix=rSVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
roi_time += @elapsed rYinf = rSVD_op_inf.C[1:1, 1:r] * rXinf;

# Compute errors
rSVD_PE, rSVD_ISE, rSVD_IOE, rSVD_OSE, rSVD_OOE = LnL.compute_all_errors(Xtest, Ytest, rXinf, rYinf, rXinf, rYinf, rSVD_Vr);
printstyled("The total randomized svd time is: ", rsvd_time, "\n")
printstyled("The operator inference time is: ", roi_time, "\n")
printstyled("The randomized svd data error is: ", norm(X - rSVD_Vr * rSVD_Vr'* X),"\n")
printstyled("The randomized svd state error is: ", rSVD_OSE, "\n")

The total randomized svd time is: 0.034414834
The operator inference time is: 7.876200457000001
The randomized svd data error is: 0.6067586744866879
The randomized svd state error is: 0.005493834470899789
